# Este notebook implementa o treinamento do modelo sumarizador utilzando um corpus de treinamento e word embeddings pré-treinados

## Carregando o dataset

In [1]:
import pandas as pd

In [2]:
caminho_dataset = "../data/dataset_artigos_de_noticias.csv"
dataset_noticias = pd.read_csv(caminho_dataset)

In [3]:
# Exibe as 5 primeiras linhas do dataset carregado
dataset_noticias.head()

,title,text,date,category,subcategory,link
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
1,"'Decidi ser escrava das mulheres que sofrem', ...","Para Oumou Sangaré, cantora e ativista malines...",2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
3,Filme 'Star Wars: Os Últimos Jedi' ganha trail...,A Disney divulgou na noite desta segunda-feira...,2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
4,CBSS inicia acordos com fintechs e quer 30% do...,"O CBSS, banco da holding Elopar dos sócios Bra...",2017-09-10,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/10/1...


In [4]:
len(dataset_noticias)

167053

In [5]:
print("Título 3:\n", dataset_noticias['title'][3], '\n')
print("Texto 3:\n", dataset_noticias['text'][3], '\n')

Título 3:
 Filme 'Star Wars: Os Últimos Jedi' ganha trailer definitivo; assista 

Texto 3:
 A Disney divulgou na noite desta segunda-feira (9) o novo trailer de "Star Wars: Os Últimos Jedi", oitavo episódio da saga.  O trailer era aguardado pelos fãs e se tornou um dos tópicos mais comentados no Twitter no horário de seu lançamento.  Assista ao trailer de 'Os Últimos Jedi'  Assista ao trailer de 'Os Últimos Jedi'  Em "O Despertar da Força" (2015), episódio mais recente, a personagem Rey (Daisy Ridley) descobre que tem a Força e procura por Luke Skywalker (Mark Hamill) para começar seu treinamento Jedi.  A história do novo episódio continua desse ponto, e cenas do trailer mostram a relação entre Rey e Skywalker.  Com direção de Rian Johnson, o filme será lançado em 14 de dezembro no Brasil. O nono episódio, ainda sem título, encerra a nova trilogia em 20 de dezembro de 2019.  O estúdio também divulgou novo poster do filme.  Poster 



## Tokenizando por palavras

Obtendo um dicionário com todas as palavras distintas e suas respectivas frequências de ocorrência no dataset

In [6]:
from nlputils.lexical import normalization as norm
normalizer = norm.Normalizer()

In [7]:
tokens_freqs = {} # As chaves serão as palavras e os valores serão as frequências de ocorrência
for i in range(len(dataset_noticias)):
    conteudo = str(dataset_noticias['title'][i]) + '\n' + str(dataset_noticias['text'][i])
    tokens = normalizer.tokenize_words(conteudo)
    for token in tokens:
        if token in tokens_freqs:
            tokens_freqs[token] += 1
        else:
            tokens_freqs[token] = 1

In [8]:
len(tokens_freqs)

600523

Ordenando os tokens por frequência de ocorrência

In [9]:
from collections import OrderedDict
from operator import itemgetter    

tokens_freqs = OrderedDict(sorted(tokens_freqs.items(), key = itemgetter(1), reverse = True))

In [10]:
print([(t, tokens_freqs[t]) for t in list(tokens_freqs)[:10]])

[(',', 5480754), ('.', 3592625), ('de', 3549421), ('a', 2218420), ('o', 1919132), ('que', 1860195), ('e', 1710064), ('do', 1377185), ('da', 1198839), ('em', 1033886)]


## Convertendo os tokens para representações vetoriais

Carregando os vetores de palavras pré-treinados

In [9]:
from gensim.models import KeyedVectors
glove_path = "../models/glove_s100.txt"
vects_glove = KeyedVectors.load_word2vec_format(glove_path)

In [10]:
glove_embedding_weights = vects_glove.vectors * 0.1
glove_embedding_weights.std()

0.027273528

In [16]:
[token for token in list(vects_glove.vocab)[:10]]

[',', 'de', '.', 'a', 'o', 'e', 'que', 'do', 'da', 'em']

Usando o GloVe para inicializar a embedding matrix

In [21]:
import numpy as np
# gera embeddings aleatórios com a mesma escala do glove
seed=42
np.random.seed(seed)
tam_vocab = len(tokens_freqs)
dim_embedding = 100 # vetores de 100 dimensões
shape = (tam_vocab, dim_embedding)
scala = glove_embedding_weights.std()*np.sqrt(12)/2 # uniforme e não normal
embedding = np.random.uniform(low=-scala, high=scala, size=shape)
print('embedding aleatório/escala glove', scala, 'std', embedding.std())

embedding aleatório/escala glove 0.047239136678944256 std 0.027271504093472725
